### Abstract


- Roberta 진행 중 기본 baseline모델을 생성하여 진행하였음.


###  need 
- epoch 늘리기
- batch size 올려보기
- 전처리 진행하기 (중복 문장, 한국어 맞춤 토큰화) 
- self.df_data.loc[index, 'premise'] <- 데이터 들어가는 방식인 이거 찍어봐야함

### result 
- 전처리 후 
Validation...
Val loss: 81.38406313583255
Val acc:  0.8164

In [4]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

## XLM-RoBERTa-Tokenizer


### bert 분류 문제 input
<img src="https://nlp.gluon.ai/_images/bert-sentence-pair.png" height="74%" width="50%">  
 
1. **input_ids, type: tensor**  
input_ids는 토큰화 된 문장입니다. 두 문장을 토큰화 할 때는 문장 사이에 __special token__인 [SEP] 토큰을 붙여 토큰화 합니다.   
__XLM-RoBERTa__의 special token 종류는
    * [BOS]: 문장의 시작을 알리는 토큰입니다.(bos_token_id: 0) 
    * [EOS]: 문장의 끝을 알리는 토큰입니다.(eos_token_id: 2)  
    * [SEP]: 문장을 나누는 토큰입니다. (sep_token_id: 2)  
    * [PAD]: 문장의 길이를 맞춰주는 토큰입니다.(pad_token_id: 1)   

2. **attention_mask, type: tensor**  
attention_mask는 pad 토큰이 있는 자리는 0, 나머지는 1을 반환합니다.   
attention_mask의 길이는 항상 input_ids길이와 같아야 합니다. 

3. **labels, type: tensor**  
참(Entailment) 또는 거짓(Contradiction) 또는 중립(Neutral

In [5]:
# XLM-RoBERTa 토크나이저를 불러옵니다: https://huggingface.co/xlm-roberta-large
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-base'
# 만약 colab pro가 아니면 MODEL_TYPE = 'xlm-roberta-base'를 사용하세요
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

ImportError: 
XLMRobertaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment.


In [6]:
# XLM-RoBERTa vocab크기 확인
tokenizer.vocab_size

250002

In [7]:
# XLM-RoBERTa vocab 확인
list(tokenizer.get_vocab())[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', '▁', 's', '▁de', '-']

In [8]:
# 문장 1개 토큰화
MAX_LEN = 10 # 문장의 길이가 max_len보다 크면 안 됩니다. 
# bert의 최대 max length는 512입니다. '안녕하세요'-> len:5 가 아니라, vocab 단어의 토큰 개수 입니다.  '안녕하세요'(107687) -> len:1

sentence1 = '안녕하세요'

encoded_dict = tokenizer.encode_plus(
            sentence1,                
            add_special_tokens = True, 
            truncation=True,
            max_length = MAX_LEN,     
            pad_to_max_length = True, # token_ids 길이를 max_len만큼 맞춰줍니다. 
            return_attention_mask = True,  
            return_tensors = 'pt' # pytorch 텐서를 반환합니다. 
       )

encoded_dict

{'input_ids': tensor([[     0, 107687,      2,      1,      1,      1,      1,      1,      1,
              1]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [7]:
# 문장 2개 토큰화
MAX_LEN = 12 

sentence1 = '안녕하세요'
sentence2 = '데이콘입니다.'

encoded_dict = tokenizer.encode_plus(
            sentence1, sentence2,      
            add_special_tokens = True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,   
            return_tensors = 'pt' 
       )


encoded_dict

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[     0, 107687,      2,      2,      6,  57991,  58470,   5826,      5,
              2,      1,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [9]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(att_mask)

tensor([     0, 107687,      2,      1,      1,      1,      1,      1,      1,
             1])
tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0])


In [210]:
# 토큰 디코드
tokenizer.decode(input_ids,
                skip_special_tokens=False)

'<s> 안녕하세요</s></s> 데이콘입니다.</s><pad><pad>'

In [211]:
#truncation : max_length 길이를 (이어서) 만약 넘어갔다면, 넘어간 이후를 잘라내버리는 방법을 사용
MAX_LEN = 3

sentence = '안녕하세요 히히 나는'

encoded_dict = tokenizer.encode_plus(
            sentence,                
            add_special_tokens = True, 
            truncation=True,
            max_length = MAX_LEN,     
            pad_to_max_length = True, 
            return_attention_mask = True,  
            return_tensors = 'pt' 
       )


encoded_dict

{'input_ids': tensor([[     0, 107687,      2]]), 'attention_mask': tensor([[1, 1, 1]])}

In [212]:
hello = torch.tensor([101351])
tokenizer.decode(hello,
                skip_special_tokens=False)

'闹'

In [ ]:
# GPU 확인
!nvidia-smi

Fri Feb 25 05:59:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |   9933MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
# device 설정
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [11]:
# 데이터 불러오기
path='/content/drive/MyDrive/데이콘 문장 관계/data'
train = pd.read_csv("/content/drive/MyDrive/AI/dataset/한국어 문장 관계 분류/train_data.csv")
test = pd.read_csv("/content/drive/MyDrive/AI/dataset/한국어 문장 관계 분류/test_data.csv")
submission = pd.read_csv("/content/drive/MyDrive/AI/dataset/한국어 문장 관계 분류/sample_submission.csv")

# label 인코딩
label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}
train['label_num'] = [label_dict[i] for i in train.label]
train.head()

,index,premise,hypothesis,label,label_num
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,1
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,1
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,0
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,2
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,2


In [12]:
train['label_num']

0        1
1        1
2        0
3        2
4        2
        ..
24993    2
24994    0
24995    1
24996    0
24997    2
Name: label_num, Length: 24998, dtype: int64

In [13]:
# 중복 확인 후 제거
col = ['premise', 'hypothesis']

print('중복 제거 전 train_data :', len(train)-len(train[col].value_counts()),"개")

train.drop_duplicates(['premise'],inplace = True)

print('중복 제거 후 train_data :',len(train)-len(train[col].value_counts()),"개")


중복 제거 전 train_data : 2 개
중복 제거 후 train_data : 0 개


In [14]:
# 특수문자 제외

train['premise'] = train['premise'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train['hypothesis'] = train['hypothesis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

test['premise'] = test['premise'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['hypothesis'] = test['hypothesis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [282]:
# 한국어 tokenizer
from konlpy.tag import Okt

#객체 생성
tokenizer = Okt()

X_train_pre = []
X_train_hy = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도', '를','으로','자','에','와','한','하다','을','에서','해서']

for row in train.iterrows() :
    premise = tokenizer.morphs(row[1]['premise'])
    premise = [word for word in premise if not word in stopwords] #불용어 제거  
    

    hypothesis =  tokenizer.morphs(row[1]['hypothesis'])
    hypothesis = [word for word in hypothesis if not word in stopwords] #불용어 제거
    
    X_train_pre.append(premise)
    X_train_hy.append(hypothesis)

KeyboardInterrupt: ignored

In [15]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.1)
print(len(train_dataset))
print(len(val_dataset))

7548
839


In [16]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]
val_dataset

,index,premise,hypothesis,label,label_num
0,1951,코레일 사장 최연혜가 새누리당 지도부에 인사청탁을 했다는 사실이 알려져 논란이 되고 있다,코레일 사장 최연혜는 여러 차례 검찰의 수사를 받았다,neutral,2
1,5622,부모님은 포르투칼어만 가능하신데 정이 넘치는 시골 할머니 할아버지같습니다,부모님은 영어를 할 줄 모르십니다,entailment,0
2,11290,중기부와 행사주관기관 지자체 공동으로 방역대응반을 구성해 발열체크 마스크 착용 거리...,방역대응반은 총 명으로 구성된다,neutral,2
3,5687,세텐자는 남부 동맹의 군자금 만 달러가 빼돌려졌다는 정보를 입수하게 된다,세텐자는 군자금에 대한 어떠한 소식도 모른다,contradiction,1
4,16951,그리고 위치도 여행하기 좋은 위치였어요,장점은 위치가 좋다는 것 뿐이었어요,neutral,2
...,...,...,...,...,...
834,12417,최근에는 새로운 서핑 장소로도 각광받고 있다,옛부터 유서깊은 오래된 서핑 장소다,contradiction,1
835,2209,서점과 책방 골목 순례도 영상으로 가능하다,영상은 분 길이로 구성되어있다,neutral,2
836,4665,누리 빌게 제일란의 최고의 감동 영화 터키의 이국적 정서,터키의 이국적 정서를 보여주는 영화,entailment,0
837,4796,친절한 호스트와 너무너무 예쁜 아파트입니다,예쁜 아파트와 잘 어울리는 호스트입니다,neutral,2


In [17]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']


        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [18]:
# 모델 하이퍼파라미터

L_RATE = 1e-5 
MAX_LEN = 256 

BATCH_SIZE = 32 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

NUM_CORES

4

In [19]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [20]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [21]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)



# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)


del train
del test
del train_data
del test_data
del val_data



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))
1407

236
27
53


1407

In [ ]:
# 학습 횟수
NUM_EPOCHS=5

# loss값 저장
loss_values = []

# 학습 시작
for epoch in range(NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        loss = outputs[0]
        
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값
        preds = outputs[1]


        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # 모델 저장
    torch.save(model.state_dict(), 'epoch:{}_model.pt'.format(epoch))
    
    # 메모리 관리
    gc.collect()


======== Epoch 1 / 5 ========
Training...
Train loss: 252.7017360329628

Validation...
Val loss: 23.68068754673004
Val acc:  0.6066746126340882

======== Epoch 2 / 5 ========
Training...
Train loss: 191.4610823392868

Validation...
Val loss: 19.663774073123932
Val acc:  0.733015494636472

======== Epoch 3 / 5 ========
Training...
Train loss: 154.83232697844505

Validation...
Val loss: 17.252118855714798
Val acc:  0.7449344457687723

======== Epoch 4 / 5 ========
Training...
